## Daily Pivot Weather

In [ ]:
%reset -f
import sys, os
sys.path.append('/home/seyedkazemi/codes/HPCRoomModel/mskhelper/')
import pandas as pd, numpy as np
import logging
import datetime, pytz
import mohsenutils, data_provider, m100_preprocessing_helper
import imp 
imp.reload(mohsenutils)
imp.reload(data_provider)
imp.reload(m100_preprocessing_helper)


pub = 'weather_pub'

for metric in ['temp']:
    
    
    print('__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'+metric+'__')
    logging.info(str('__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'))
    log_file_name = 'CRAC_log_raw_data_cook_'+datetime.datetime.strftime(datetime.datetime.now(pytz.timezone('Europe/Rome')), 
                                                                      '%Y-%m-%d %H:%M:%S').replace(':','-').replace(' ','_')+'.log'
    logging.basicConfig(filename=log_file_name,level=logging.INFO, format='%(levelname)s:%(asctime)s:%(lineno)d:%(message)s')


    path = os.path.join('/home/seyedkazemi/dataset/Marconi100_backup/',pub,metric,'')

    if not os.path.isdir(path+'/'+metric+'_done/'):
        os.makedirs(path+'/'+metric+'_done/')

    if not os.path.isdir(path+'/'+metric+'_pivot/'):
        os.makedirs(path+'/'+metric+'_pivot/')    

    files_list = sorted([file for file in os.listdir(path) if '.csv.gz' in file])
    for file in files_list:
        print(file)
        logging.info(str(file))
        
        data = pd.read_csv(path+file, compression='gzip', index_col='timestamp')
        
        data.loc[data['value'] == 'Unavailable', 'value'] = np.nan
        data.loc[data['value'] == 'unknown', 'value'] = np.nan
        data['value'] = data['value'].astype(float)


        
        print('Data shape: ', data.shape)
        logging.info('Data shape: '+ str(data.shape))


        data = data.pipe(m100_preprocessing_helper.remove_Unnamed, log=True).\
        pipe(m100_preprocessing_helper.unique_val, log=True).\
        pipe(m100_preprocessing_helper.pvt, index='timestamp', freq='10T', columns='type', values='value', log=True)

        data.to_csv(path+'/'+metric+'_pivot/'+file.split('.')[0]+'.csv.gz', compression='gzip')
        os.replace(path+file, path+metric+'_done/'+file)
        print(50*'#')
        logging.info(str(50*'#'))
    
    
   

## Create One Big file

In [4]:
%reset -f
import sys, os
sys.path.append('/home/seyedkazemi/codes/HPCRoomModel/mskhelper/')
import pandas as pd, numpy as np
import glob
import logging
import datetime, pytz
import mohsenutils
import imp 
import m100_preprocessing_helper
imp.reload(m100_preprocessing_helper)



pth = '/home/seyedkazemi/dataset/Marconi100_backup/'

m100_preprocessing_helper.concat_multiple_csvs(csvdir=os.path.join(pth, 'weather_pub', 'temp', 'temp_pivot'), 
                                               big_csv_path_name=os.path.join(pth,'weather_pub','temp', 'temp_pivot','temp_big.csv.gz'),
                                               freq='10T')




['/home/seyedkazemi/dataset/Marconi100_backup/weather_pub/temp/temp_pivot/temp_2021-04-22_00-00-00_to_2021-04-23_00-00-00.csv.gz', '/home/seyedkazemi/dataset/Marconi100_backup/weather_pub/temp/temp_pivot/temp_2021-05-04_00-00-00_to_2021-05-05_00-00-00.csv.gz', '/home/seyedkazemi/dataset/Marconi100_backup/weather_pub/temp/temp_pivot/temp_2021-05-09_00-00-00_to_2021-05-10_00-00-00.csv.gz', '/home/seyedkazemi/dataset/Marconi100_backup/weather_pub/temp/temp_pivot/temp_2021-05-10_00-00-00_to_2021-05-11_00-00-00.csv.gz', '/home/seyedkazemi/dataset/Marconi100_backup/weather_pub/temp/temp_pivot/temp_2021-05-05_00-00-00_to_2021-05-06_00-00-00.csv.gz', '/home/seyedkazemi/dataset/Marconi100_backup/weather_pub/temp/temp_pivot/temp_2021-05-02_00-00-00_to_2021-05-03_00-00-00.csv.gz', '/home/seyedkazemi/dataset/Marconi100_backup/weather_pub/temp/temp_pivot/temp_2021-05-08_00-00-00_to_2021-05-09_00-00-00.csv.gz', '/home/seyedkazemi/dataset/Marconi100_backup/weather_pub/temp/temp_pivot/temp_2021-05-01_

,current,daily_forecast,hourly_forecast
timestamp,,,
2021-04-20 13:00:00,NaN,16.76,NaN
2021-04-20 13:10:00,NaN,NaN,NaN
2021-04-20 13:20:00,NaN,NaN,NaN
2021-04-20 13:30:00,NaN,NaN,NaN
2021-04-20 13:40:00,NaN,NaN,NaN
...,...,...,...
2021-05-10 23:20:00,16.63,NaN,NaN
2021-05-10 23:30:00,16.50,NaN,NaN
2021-05-10 23:40:00,16.25,NaN,NaN


## COPY Temp DATA

In [5]:
from shutil import copyfile
copyfile('/home/seyedkazemi/dataset/Marconi100_backup/weather_pub/temp/temp_pivot/temp_big.csv.gz', '/home/seyedkazemi/dataset/M100_Complete/temp_big.csv.gz')


'/home/seyedkazemi/dataset/M100_Complete/temp_big.csv.gz'